# Health Coding Challenge

### Cameron Goldbeck

Context: Several cancer drugs are being provided to patients, Drugs A,B,C,D, some of which are given in tandum.

Goal: Understand the how drug therapies are given out and which may seem more effective for certain groups.

In [4]:
# Load libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from functools import reduce

In [5]:
# Load patient diagnosis data
d = pd.read_csv('https://raw.githubusercontent.com/cgoldbeck/Datasets/master/Patient_Diagnosis.csv')
d['diagnosis_date'] = pd.to_datetime(d['diagnosis_date'])
d.head()

,patient_id,diagnosis_date,diagnosis_code,diagnosis
0,2120,2010-01-09,174.1,Breast Cancer
1,2720,2010-01-09,174.1,Breast Cancer
2,2038,2010-01-21,174.9,Breast Cancer
3,2238,2010-01-21,174.9,Breast Cancer
4,2175,2010-02-17,174.7,Breast Cancer


In [6]:
# Load patient treatment data
t = pd.read_csv('https://raw.githubusercontent.com/cgoldbeck/Datasets/master/Patient_Treatment.csv')
t['treatment_date'] = pd.to_datetime(t['treatment_date'])
t.head()

,patient_id,treatment_date,drug_code
0,2720,2010-01-20,B
1,2238,2010-01-21,B
2,2120,2010-01-23,B
3,2038,2010-01-24,A
4,2120,2010-01-24,A


In [7]:
# identify how many entries (unique and multiple) there are in the data and why there are multiples
ids = d['patient_id'].value_counts()
id_mult = ids[ids > 1]
d.loc[d['patient_id'].isin(id_mult.index)]

,patient_id,diagnosis_date,diagnosis_code,diagnosis
18,3449,2011-09-09,153.5,Colon Cancer
19,3449,2011-09-09,153.4,Colon Cancer
20,3757,2011-10-11,174.1,Breast Cancer
21,3757,2011-10-17,153.4,Colon Cancer
22,3757,2011-10-17,153.5,Colon Cancer
24,4256,2011-11-07,174.5,Breast Cancer
25,4256,2011-11-07,174.8,Breast Cancer
29,4354,2012-02-04,174.8,Breast Cancer
30,4354,2012-02-04,174.5,Breast Cancer
31,4374,2012-03-20,174.5,Breast Cancer


In [8]:
d_uni = d.sort_values(by=['patient_id', 'diagnosis_date'])
d_uni = d_uni.drop_duplicates(subset='patient_id', keep='first')
print("Original Size: ", d.shape, "Unique Size: ", d_uni.shape)

Original Size:  (57, 4) Unique Size:  (47, 4)


There are 47 unique individuals in the data with 9 of those having multiple diagnoses, either both forms of cancer or multiple codes for a single type.

### Question 1:
First, the clinic would like to know the distribution of cancer types across their patients. Please provide the clinic with this information.

In [9]:
# So we do not double count, we use the unique dataset
diagnosis = d_uni['diagnosis'].value_counts()
diagnosis = diagnosis / diagnosis.sum()

In [10]:
diagnosis

Breast Cancer    0.680851
Colon Cancer     0.319149
Name: diagnosis, dtype: float64

### Answer 1:

There are multiple entries for some individuals that represent slightly different diagnosis codes for the same overall cancer diagnosis, therefore we first need to filter down to one row per individual to get a distribution of cancer types. Overall, about 68% of patients have breast cancer while the remaining 32% of patients have colon cancer. 

### Question 2:
The clinic wants to know how long it takes for patients to start therapy after being diagnosed, which they consider to be helpful in understanding the quality of care for the patient. How long after being diagnosed do patients start treatment?

In [11]:
# create a df with the first treatment entry for each patient
t_uni = t.sort_values(by=['patient_id', 'treatment_date'])
t_uni = t_uni.drop_duplicates(subset='patient_id', keep='first')
print("Original Size: ", t.shape, "Unique Size: ", t_uni.shape)

Original Size:  (1096, 3) Unique Size:  (46, 3)


In [12]:
# check to see if there are patients in the treatment dataset not in diagnosis
set(d_uni['patient_id']) - set(t_uni['patient_id'])

{4256}

In [13]:
d.loc[d['patient_id'] == 4256, :]

,patient_id,diagnosis_date,diagnosis_code,diagnosis
24,4256,2011-11-07,174.5,Breast Cancer
25,4256,2011-11-07,174.8,Breast Cancer


In [14]:
t.loc[t['patient_id'] == 4256, :]  # not in the treatment data

,patient_id,treatment_date,drug_code


In [15]:
# merge the diagnosis and first treatment date datasets
d_t = d_uni.merge(t_uni, left_on='patient_id', right_on='patient_id')
d_t['time_to_treatment'] = d_t['treatment_date'] - d_t['diagnosis_date']
d_t.head()

,patient_id,diagnosis_date,diagnosis_code,diagnosis,treatment_date,drug_code,time_to_treatment
0,2038,2010-01-21,174.9,Breast Cancer,2010-01-24,A,3 days
1,2120,2010-01-09,174.1,Breast Cancer,2010-01-23,B,14 days
2,2175,2010-02-17,174.7,Breast Cancer,2010-02-21,A,4 days
3,2238,2010-01-21,174.9,Breast Cancer,2010-01-21,B,0 days
4,2407,2010-06-13,174.9,Breast Cancer,2010-06-19,A,6 days


In [16]:
print('Average time to treatment (in days): ', d_t['time_to_treatment'].mean().days)
print('Range of time to treatment (in days): ', d_t['time_to_treatment'].min().days, ', ', d_t['time_to_treatment'].max().days)

Average time to treatment (in days):  11
Range of time to treatment (in days):  -3 ,  304


In [17]:
d_t['time_to_treatment'][d_t['time_to_treatment'] < np.timedelta64(0, 'D')]

43   -3 days
Name: time_to_treatment, dtype: timedelta64[ns]

In [18]:
d_t2 = d_t
d_t2.loc[d_t['time_to_treatment'] < np.timedelta64(0, 'D'), 'time_to_treatment'] = np.timedelta64(0, 'D')
print('Average time to treatment (in days): ', d_t2['time_to_treatment'].mean().days)
print('Range of time to treatment (in days): ', d_t2['time_to_treatment'].min().days, ', ', d_t2['time_to_treatment'].max().days)

Average time to treatment (in days):  11
Range of time to treatment (in days):  0 ,  304


### Answer 2:

Using the earliest date of treatment reported, we calculate the difference in days between when patients are diagnosed and when their earliest treatment is reported. We find the means number of days to treatment is 11 while the raw range is -3 days to 304 days. Obviously this negative time to treatment is not possible and is most likely a quality issue with the date (e.g. entry). To account for this, we can replace negative values with a 0, which may most closely reflect their quick time to treatment. By doing this we find the same average of 11 days to treatment and the range is then 0 days to 304 days to treatment.

### Quesiton 3: 

Which treatment regimens do you think would be indicated to be used as first-line of treatment for breast cancer? What about colon cancer? 

In [19]:
t.head()

,patient_id,treatment_date,drug_code
0,2720,2010-01-20,B
1,2238,2010-01-21,B
2,2120,2010-01-23,B
3,2038,2010-01-24,A
4,2120,2010-01-24,A


In [20]:
# gather treatment types individually
t_a = t.loc[t['drug_code'] == 'A', :]
t_b = t.loc[t['drug_code'] == 'B', :]
t_c = t.loc[t['drug_code'] == 'C', :]
t_d = t.loc[t['drug_code'] == 'D', :]

In [21]:
# sort by treatment data so we know which drug was taken first if multiple were taken
t_a = t_a.sort_values(by=['patient_id', 'treatment_date']).drop_duplicates(subset='patient_id', keep='first')
t_b = t_b.sort_values(by=['patient_id', 'treatment_date']).drop_duplicates(subset='patient_id', keep='first')
t_c = t_c.sort_values(by=['patient_id', 'treatment_date']).drop_duplicates(subset='patient_id', keep='first')
t_d = t_d.sort_values(by=['patient_id', 'treatment_date']).drop_duplicates(subset='patient_id', keep='first')

In [22]:
# combine all treatment sets
hold = [t_a, t_b, t_c, t_d]
ts = reduce(lambda  left,right: pd.merge(left,right,on=['patient_id'], how='outer'), hold).fillna(np.nan)
ts.columns = ['patient_id', 'treatment_date_A', 'drug_code_A', 'treatment_date_B', 'drug_code_B', 'treatment_date_C', 
                 'drug_code_C', 'treatment_date_D', 'drug_code_D']

In [23]:
ts.head()

,patient_id,treatment_date_A,drug_code_A,treatment_date_B,drug_code_B,treatment_date_C,drug_code_C,treatment_date_D,drug_code_D
0,2038,2010-01-24,A,2010-01-24,B,NaT,NaN,NaT,NaN
1,2120,2010-01-24,A,2010-01-23,B,NaT,NaN,NaT,NaN
2,2175,2010-02-21,A,2010-02-21,B,NaT,NaN,NaT,NaN
3,2407,2010-06-19,A,2010-06-19,B,NaT,NaN,NaT,NaN
4,2425,2010-12-19,A,2010-12-19,B,NaT,NaN,NaT,NaN


In [24]:
# identify first line treatment comparing dates of all treatment dates
ts['first_line_date'] = ts.loc[:, ['treatment_date_A', 'treatment_date_B', 'treatment_date_C', 'treatment_date_D']].min(axis=1)
ts.loc[ts['treatment_date_A'] != ts['first_line_date'], 'drug_code_A'] = np.nan
ts.loc[ts['treatment_date_B'] != ts['first_line_date'], 'drug_code_B'] = np.nan
ts.loc[ts['treatment_date_C'] != ts['first_line_date'], 'drug_code_C'] = np.nan
ts.loc[ts['treatment_date_D'] != ts['first_line_date'], 'drug_code_D'] = np.nan
ts['first_line'] = ts[['drug_code_A', 'drug_code_B', 
                       'drug_code_C', 'drug_code_D']].apply(lambda x: 
                                                            None if x.isnull().all() else ''.join(x.dropna()), axis=1)
ts = ts.merge(d_uni[['patient_id', 'diagnosis']], left_on='patient_id', right_on='patient_id')
ts.head()

,patient_id,treatment_date_A,drug_code_A,treatment_date_B,drug_code_B,treatment_date_C,drug_code_C,treatment_date_D,drug_code_D,first_line_date,first_line,diagnosis
0,2038,2010-01-24,A,2010-01-24,B,NaT,NaN,NaT,NaN,2010-01-24,AB,Breast Cancer
1,2120,2010-01-24,NaN,2010-01-23,B,NaT,NaN,NaT,NaN,2010-01-23,B,Breast Cancer
2,2175,2010-02-21,A,2010-02-21,B,NaT,NaN,NaT,NaN,2010-02-21,AB,Breast Cancer
3,2407,2010-06-19,A,2010-06-19,B,NaT,NaN,NaT,NaN,2010-06-19,AB,Breast Cancer
4,2425,2010-12-19,A,2010-12-19,B,NaT,NaN,NaT,NaN,2010-12-19,AB,Breast Cancer


In [25]:
# list first line treatment for each cancer type
drugs_bc, counts_bc = np.unique(ts.loc[ts['diagnosis'] == 'Breast Cancer', 'first_line'], return_counts=True)
percent_bc = counts_bc/counts_bc.sum()

drugs_cc, counts_cc = np.unique(ts.loc[ts['diagnosis'] == 'Colon Cancer', 'first_line'], return_counts=True)
percent_cc = counts_cc/counts_cc.sum()

print('First line treatment breakdown for breast cancer', '\n', np.asarray((drugs_bc, counts_bc, percent_bc)).T)
print('\n')
print('First line treatment breakdown for colon cancer', '\n', np.asarray((drugs_cc, counts_cc, percent_cc)).T)

First line treatment breakdown for breast cancer 
 [['A' 4 0.12903225806451613]
 ['AB' 17 0.5483870967741935]
 ['B' 7 0.22580645161290322]
 ['C' 3 0.0967741935483871]]


First line treatment breakdown for colon cancer 
 [['AB' 4 0.26666666666666666]
 ['B' 4 0.26666666666666666]
 ['C' 3 0.2]
 ['D' 4 0.26666666666666666]]


### Answer 3:

It appears that for breast cancer the most common first like treatment is the combination of Drugs A and B with about 55% of breast cancer patients receiving them as their first treatment. This is followed by only Drug B at 23%, only Drug A at 13%, and finally Drug C with about 10% of patients receiving this drug first.

For colon cancer the results are more spread. We find that equal number of colon cancer patients received Drug A and B in combinaiton, Drug B alon, and Drug D, all at about 27% of patients. This is followed by Drug C with 20% of patients receiving this drug first.

### Question 4:

Do the patients taking Regimen A vs. Regimen B as first-line therapy for breast cancer vary in terms of duration of therapy? 

In [26]:
# gather patients for each type of first line treatment and cancer type
As = ts['patient_id'][(ts['first_line'] == 'A') & (ts['diagnosis'] == 'Breast Cancer')]
Bs = ts['patient_id'][(ts['first_line'] == 'B') & (ts['diagnosis'] == 'Breast Cancer')]

In [27]:
# gather furst and last days of treatment for both drugs
t = t.sort_values(by=['patient_id', 'treatment_date'])
t_bi1 = t.drop_duplicates(subset='patient_id', keep='first')
t_bi2 = t.drop_duplicates(subset='patient_id', keep='last')
t_bi3 = t_bi1.merge(t_bi2, left_on='patient_id', right_on='patient_id')
t_bi3['treatment_lenght'] = t_bi3['treatment_date_y'] - t_bi3['treatment_date_x']

In [28]:
t_bi_A = t_bi3.loc[t_bi3['patient_id'].isin(As), :]
t_bi_B = t_bi3.loc[t_bi3['patient_id'].isin(Bs), :]

In [29]:
print('The mean number of days of treatment for first-line therapy A: ', '\n', t_bi_A['treatment_lenght'].mean().days)
print('Range: ', t_bi_A['treatment_lenght'].min().days, ', ', t_bi_A['treatment_lenght'].max().days)
print('\n')
print('The mean number of days of treatment for first-line therapy B: ', '\n', t_bi_B['treatment_lenght'].mean().days)
print('Range: ', t_bi_B['treatment_lenght'].min().days, ', ', t_bi_B['treatment_lenght'].max().days)

The mean number of days of treatment for first-line therapy A:  
 80
Range:  74 ,  93


The mean number of days of treatment for first-line therapy B:  
 191
Range:  0 ,  1001


### Answer 4:

It appears that those who are given Drug B as their first line therapy spend over double the amount of days in treatment. For drug A the mean number of days is 80 while for drug B it is 191 days. We should note that on top of a small sample for both drugs, drug A has a very narrow range 74 days to 93 days while drug B is rather large ranging from 0 days to over 1000 days so this limits the inferance we can make about his average.

# General Questions
Thoughts going further about what else we want to know about this data and these patients.

## Question 1:

When presented with a new dataset or database, what steps do you generally take to evaluate it prior to working with it?

### Answer 1:

With a new data set, it is important to get an idea of the structure of the data. Does it appear to be cross-sectional, longitudinal, long/wide. Are there any duplicates in any sense, should this be allowed? I think it is important to get an idea of what the underlying data looks like before doing any work on it so that I have a concept of what results I should be getting. Further, understanding the values in the data. Are things categorical, do I need a data dictionary, what units are numbers measured in. 

## Question 2:

Based on the information provided above and the attached dataset, what three questions would you like to understand prior to conducting any analysis of the data?

### Answer 2:

1) For those receiving Drugs A and B for first line therapy, is one of the two drugs more influential regarding outcomes?
2) There is a person reporting treatment before diagnosis, what are the correct dates for this person?
3) Is it best to collapse multiple diagnosis of the same cancer type for an individual?